## Import das biblitecas 

In [115]:
import pandas as pd
import smtplib

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

## Importando o arquivo Excel
Adicionei uma coluna chamada Semana

In [116]:
vendas = pd.read_excel("vendas.xlsx")
vendas['Semana'] = vendas['Data'].dt.isocalendar().week

## DataFrame dos gerentes

In [117]:
gerentes = {
    'Login' : ['carlosfsp','gustavasc','jorgelp'],
    'Email' : ['carlosfsp@dcc.ufrj.br','gustavasc@dcc.ufrj.br','jorgelp@dcc.ufrj.br'],
    'Loja' : ['Shopping Barra', 'Ribeirão Shopping', 'Shopping Recife']
}

df_gerentes = pd.DataFrame(gerentes)

In [118]:
df_gerentes.head()

,Login,Email,Loja
0,carlosfsp,carlosfsp@dcc.ufrj.br,Shopping Barra
1,gustavasc,gustavasc@dcc.ufrj.br,Ribeirão Shopping
2,jorgelp,jorgelp@dcc.ufrj.br,Shopping Recife


## DataFrame de faturamento
Aqui criei um DF de faturamente de todas as lojas, agrupados por semana.

In [130]:
faturamento = vendas.groupby(['ID Loja','Semana']).agg(
    faturamento_total = ('Valor Final', 'sum'),
    Produto_vendidos = ('Quantidade', 'sum'),
    Ticket_medio = ('Valor Unitário', 'median')
).reset_index().sort_values(by='Semana', ascending=False)

faturamento['faturamento_total'] = faturamento['faturamento_total'].astype(float)

In [133]:
faturamento.head()

,ID Loja,Semana,faturamento_total,Produto_vendidos,Ticket_medio,faturamento_total_BRL,Ticket_medio_BRL
1299,Shopping Vila Velha,52,15849.0,94,124.0,R$ 15849.00,R$ 124.00
1143,Shopping Recife,52,23082.0,106,155.0,R$ 23082.00,R$ 155.00
675,Salvador Shopping,52,23207.0,107,145.0,R$ 23207.00,R$ 145.00
831,Shopping Center Leste Aricanduva,52,12941.0,73,152.0,R$ 12941.00,R$ 152.00
415,Parque Dom Pedro Shopping,52,11370.0,55,116.0,R$ 11370.00,R$ 116.00


## Função que escolhe a loja a partir do login do gerente

In [120]:
def loja_gerente(login):
    dados_login = df_gerentes.loc[df_gerentes['Login'] == login].copy()
    
    if not dados_login.empty:
        loja_id = dados_login['Loja'].values[0]
        loja_do_gerente = faturamento.loc[faturamento['ID Loja'] == loja_id].copy()
        return loja_do_gerente
    else:
        raise ValueError("Login não encontrado")
    

## Função para formatar colunas em BRL

In [121]:
def convert_to_currance(value):
    return f'R$ {value:.2f}'.replace(',',',')

Criando arquivo de faturamento geral

In [136]:
faturamento['faturamento_total_BRL'] = faturamento['faturamento_total'].apply(convert_to_currance)
faturamento['Ticket_medio_BRL'] = faturamento['Ticket_medio'].apply(convert_to_currance)
faturamento.to_excel(f'faturamento_geral.xlsx')

## Criando DataFrame da loja pedida pelo gerente

In [134]:
login = input("Login: ")
loja = loja_gerente(login)
loja['Ticket_medio_BRL'] = faturamento['Ticket_medio_BRL']
loja['faturamento_total_BRL'] = faturamento['faturamento_total_BRL']

In [123]:
loja.to_excel(f'faturamento_loja_do_{login}.xlsx')

In [135]:
loja.head()

,ID Loja,Semana,faturamento_total,Produto_vendidos,Ticket_medio,faturamento_total_BRL,Ticket_medio_BRL
727,Shopping Barra,52,10449.0,53,150.0,R$ 10449.00,R$ 150.00
726,Shopping Barra,51,28126.0,137,181.0,R$ 28126.00,R$ 181.00
725,Shopping Barra,50,23691.0,136,118.0,R$ 23691.00,R$ 118.00
724,Shopping Barra,49,36140.0,225,130.0,R$ 36140.00,R$ 130.00
723,Shopping Barra,48,32845.0,143,177.0,R$ 32845.00,R$ 177.00


## Declarando endereções de email

In [160]:
nome_loja = df_gerentes.loc[df_gerentes['Login'] == login, 'Loja'].values
email_fonte = "cs134922@gmail.com"
email_destino = df_gerentes.loc[df_gerentes['Login'] == login, 'Email'].values

senha_fonte = 'coloque sua senha aqui'

## Envio de Email para gerentes

In [161]:
def enviar_email_gerente(nome_loja, email_destino, email_fonte, senha_fonte):
    print(f'Você escolheu enviar o relatório para {login} ')
    
    # Verificar se existem resultados para nome_loja e email_destino
    if len(nome_loja) > 0 and len(email_destino) > 0:
        nome_loja = nome_loja[0]  # Obter o primeiro valor de nome_loja
        email_destino = email_destino[0]  # Obter o primeiro valor de email_destino
        
        try:
            msg = MIMEMultipart()
            
            msg['From'] = email_fonte
            msg['To'] = email_destino
            msg['Subject'] = f'Relatório da loja {nome_loja}'
            
            body = f"\nSegue em Anexo o relatório de vendas da loja {nome_loja}.\n\n"
            msg.attach(MIMEText(body, 'plain'))
            
            # Nome do arquivo de anexo
            filename = f'faturamento_loja_do_{login}.xlsx'
            attachment = open(filename, 'rb')
            
            part = MIMEBase('application', 'octet-stream')
            part.set_payload(attachment.read())
            encoders.encode_base64(part)
            part.add_header('Content-Disposition', f'attachment; filename={filename}')
            
            msg.attach(part)
            attachment.close()
            
            # Conectar ao servidor de email
            server = smtplib.SMTP('smtp.gmail.com', 587)
            server.starttls()
            
            # Login no servidor (troque 'SUA_SENHA_AQUI' pela senha correta)
            server.login(email_fonte, f'{senha_fonte}')
            
            # Enviar email
            text = msg.as_string()
            server.sendmail(email_fonte, email_destino, text)
            server.quit()
            
            print("Email enviado com sucesso!")
        
        except Exception as e:
            print(f"\nErro ao enviar email: {e}")
    else:
        print("Login não encontrado ou sem email associado.")

## Envio de email para Diretoria

In [162]:
def enviar_email_diretoria(email_destino, email_fonte, senha_fonte):
        print("Você escolheu enviar o relatório geral para a diretoria.")
        
        email_destino = email_destino[0]  # Obter o primeiro valor de email_destino
        
        try:
            msg = MIMEMultipart()
            
            msg['From'] = email_fonte
            msg['To'] = email_destino
            msg['Subject'] = f'Relatório Geral de Vendas'
            
            body = "\nSegue em Anexo o relatório geral de vendas.\n\n"
            msg.attach(MIMEText(body, 'plain'))
            
            # Nome do arquivo de anexo
            filename = "faturamento_geral.xlsx"
            attachment = open(filename, 'rb')
            
            part = MIMEBase('application', 'octet-stream')
            part.set_payload(attachment.read())
            encoders.encode_base64(part)
            part.add_header('Content-Disposition', f'attachment; filename={filename}')
            
            msg.attach(part)
            attachment.close()
            
            # Conectar ao servidor de email
            server = smtplib.SMTP('smtp.gmail.com', 587)
            server.starttls()
            
            # Login no servidor (troque 'SUA_SENHA_AQUI' pela senha correta)
            server.login(email_fonte, f'{senha_fonte}')
            
            # Enviar email
            text = msg.as_string()
            server.sendmail(email_fonte, email_destino, text)
            server.quit()
            
            print("Email enviado com sucesso!")
        
        except Exception as e:
            print(f"\nErro ao enviar email: {e}")

## Menu para escolher qual envio fazer

In [163]:
def menu():    
    while True:
        print("\nMenu:")
        print("1. Executar enviar_email_gerente")
        print("2. Executar enviar_email_diretoria")
        print("0. Sair")
        
        opcao = input("Escolha uma opção: \n"
                      "1 - enviar_email_gerente\n"
                      "2 - enviar_email_diretoria\n"
                      "0 - Sair\n")

        if opcao == '1':
            enviar_email_gerente(nome_loja, email_destino, email_fonte, senha_fonte)
        elif opcao == '2':
            enviar_email_diretoria(email_destino, email_fonte, senha_fonte)
        elif opcao == '0':
            print("Saindo do programa.")
            break
        else:
            print("Opção inválida. Tente novamente.")

In [164]:
menu()


Menu:
1. Executar enviar_email_gerente
2. Executar enviar_email_diretoria
0. Sair
Você escolheu enviar o relatório para carlosfsp 

Erro ao enviar email: (535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials d2e1a72fcca58-71944bc7bb5sm7416427b3a.207 - gsmtp')

Menu:
1. Executar enviar_email_gerente
2. Executar enviar_email_diretoria
0. Sair
Saindo do programa.
